In [1]:
%matplotlib inline

import sys
import os
sys.path.append('C:/Users/samrari/ComputBuffer')
from my_packages.My_Geoprocess import*

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
from skimage import exposure, img_as_float, morphology

from osgeo import gdal
import osr
import xml.etree.ElementTree as ET
import os, fnmatch


In [2]:
xml_filePath = "C:/Users/samrari/ComputBuffer/Image/Sen2/S2A_MSIL1C_20170718T231141_N0205_R101_T58KFA_20170718T231219.SAFE/"
xml_fileName = "MTD_MSIL1C.xml"
gdal.ErrorReset()
ds = gdal.Open( xml_filePath + xml_fileName )

if ds is None or gdal.GetLastErrorMsg() != '':
    print('xml failed to load')
    
DataSet_md = ds.GetMetadata()
SubDataSet_md = ds.GetMetadata('SUBDATASETS')

ds = []
# Try opening the 4 subdatasets
for i in range(4):
    gdal.ErrorReset()
    ds.append(gdal.Open(SubDataSet_md['SUBDATASET_%d_NAME' % (i+1)]))
    if ds is None or gdal.GetLastErrorMsg() != '':
        print('subdatasets failed to load')
        print(SubDataSet_md['SUBDATASET_%d_NAME' % (i+1)])
src_ds = ds[0]
ds[0].RasterCount

# Define Raster Footprint

RasterWidth = src_ds.RasterXSize
RasterHeight = src_ds.RasterYSize
GT_ds = src_ds.GetGeoTransform()
min_x = GT_ds[0]
min_y = GT_ds[3] + RasterWidth*GT_ds[4] + RasterHeight*GT_ds[5] 
max_x = GT_ds[0] + RasterWidth*GT_ds[1] + RasterHeight*GT_ds[2]
max_y = GT_ds[3] 

srs_ds = osr.SpatialReference()
srs_ds.ImportFromWkt(src_ds.GetProjection())

srsLatLong = srs_ds.CloneGeogCS()
ct_ds = osr.CoordinateTransformation(srs_ds,srsLatLong)

min_E, min_N = ct_ds.TransformPoint(min_x, min_y)[:2]
max_E, max_N = ct_ds.TransformPoint(max_x, max_y)[:2]

print('Upper left corner : ', (min_E, min_N), '\nLower right corner : ', (max_E, max_N))

Upper left corner :  (165.9735606070099, -22.692427357900442) 
Lower right corner :  (167.02786117314153, -21.691015824552036)


In [3]:
RePath_xmlDirectory = 'Data_SHOM/Global'
TimeThres = 1970

Selected_xml, LoopCount, count = GetXml_byFootprint(RePath_xmlDirectory,(min_E, max_E), (min_N, max_N), FromYear=TimeThres)

In [4]:
Set , Count, failedXml = GetSurvey_byDepthFP(Selected_xml, RePath_xmlDirectory,(min_E, max_E), (min_N, max_N), max_Depth=10000)
Set[1].shape

(18468224, 2)

In [68]:
root = ET.parse(RePath_xmlDirectory+'/'+'S201406300-1.object.xml').getroot()
int(root.find('Attribute[@name="SURSTA"]').find('Value').text[0:4])

2014

In [47]:
recordSurvey('CSV/SelectReleves.csv', Set)

In [5]:
recordPosition('CSV/SelectReleves_1970.csv', Set)

In [2]:
RePath_xmlDirectory = 'Data_SHOM/Global'


In [45]:
def RecordSurveyAndYear(XML_List, RePath_xmlDirectory, path_filname):
    
    with open(path_filname, 'w') as mon_fichier:
        print('Date',';', 'Lon', ';', 'Lat', ';', 'Depth', file= mon_fichier)

    for xml in XML_List:
        # initilise
        Set_Point = []
        Set_Depth = []
        Set_xml = []
        countValidPT = []
        NoValid_xml = []

        # read current xml
        DepthPt = pd.read_csv(RePath_xmlDirectory+'/'+xml[:-10]+'ascii', delim_whitespace=True, header=None, names=['Lat','Lon','Depth'])
        # filter by depth
        DepthPt_40m = DepthPt
        # loads points
        current_pt =  DepthPt_40m[['Lon','Lat']].values
        current_depth =  DepthPt_40m['Depth'].values

        # records points
        Set_Point.extend(current_pt[:,:])
        Set_Depth.extend(current_depth[:])
        root = ET.parse(RePath_xmlDirectory+'/'+xml).getroot()
        Set_xml.extend(np.tile((root.find('Attribute[@name="SURSTA"]').find('Value').text),len(current_depth)))
        countValidPT.append(len(current_depth))  

        Set , Count, failedXml =  [np.asarray(Set_xml),  np.asarray(Set_Point), np.asarray(Set_Depth)], countValidPT, NoValid_xml

        with open(path_filname, 'a+') as mon_fichier:
            for i in range(len(Set[2])):
                print(Set[0][i], ';', Set[1][i,0], ';', Set[1][i,1], ';', Set[2][i], file= mon_fichier)



In [48]:
Selected_xml = []
for xmlFile in GetFiles("*.xml",directory=RePath_xmlDirectory, TimLim=1900):
    Selected_xml.append(xmlFile)



In [ ]:
RecordSurveyAndYear(Selected_xml, RePath_xmlDirectory, 'ShomSurveys.csv')


In [46]:
RecordSurveyAndYear(Selected_xml[0:2], RePath_xmlDirectory, 'test.csv')
